# Sentinel Satellite Image Export
## Prerequisites
### Input

* CSV with 4 columns:
    * ID
    * year of survey
    * Latitude-Coordinates
    * Longitude-Coordinates

### Execution
#### main pipeline
1. create a pandas dataframe with all the information and delete all unnecessary entries (in our case it is all entries who are from 2012 or earlier)
2. creation of a range of year in string format (yyyy-mm-dd) instead a year in integer format
3. delete all pixels which are classified as a cloud or other classifiers
4. create an image with the median
5. get the areas of interest (AOI) with the Latitude and Longitude information in the dataframe
6. export the image with predefined AOIs and dimensions
    
Google Documentary says 'To get a block of pixels of predefined size (for example a 256x256 thumbnail image) that covers a region, specify dimensions and region'

In [ ]:
import ee
import pandas as pd
import utils
import os
import functools

In [ ]:

# Trigger the authentication flow.
#ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Constants

In [ ]:
#Path of CSV-File with columns ID,year,LATITUDE,LONGITUDE
csvpath = '../dataResearch/firstSample.csv'
#name all column names
#year
year = 'year'
#Latitude and Longitude Coordinates
LATNUM = 'LATNUM'
LONGNUM= 'LONGNUM'
#ID for Filenames
surveyid = 'ID'
#Export parameters
DHS_EXPORT_FOLDER = 'geotiff_raw'
# image export parameters
PROJECTION = 'EPSG:3857'  # see https://epsg.io/3857
#1100 pixel times 10m equals 11000mx11000m image
SCALE = 10                # export resolution: 10m/px
IMAGE_DIMENSION = 5500  #radius of the wanted image in m
MAX_CLOUD_PROBABILITY = 50

In [ ]:
#This is the MaskClouds Function with the Cloudprobability Product
def maskClouds(img, MAX_CLOUD_PROBABILITY):
    """Creates a new image with information if the pixel are clouds or not (binary)

    Args:
        img (ee.image): regional image of the sentinel imagecollection
        MAX_CLOUD_PROBABILITY (int): defines the maximum probability wether a pixel is seen as a cloud or not

    Returns:
        [ee.image]: returns a new image with 1, if the pixel is not defined as clouds and 0, if the pixel is defined as a cloud
    """
    clouds = ee.Image(img.get('cloud_mask')).select('probability')
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    return img.updateMask(isNotCloud)

In [ ]:
def maskEdges(s2_img):
    return s2_img.updateMask(s2_img.select('B8A').mask().updateMask(s2_img.select('B9').mask()))

In [ ]:
#create a Dataframe to iterate over and get all neccessary information
df = pd.read_csv(csvpath)
df = df[df.year >= 2012]
for i in range(len(df)):
    #get all constants out of the Dataframe
    start_date,end_date = utils.surveyyear_to_range(df[year].iloc[i],satellitename='S2')
    lat = float(df[LATNUM].iloc[i].replace(',', '.'))
    lon = float(df[LONGNUM].iloc[i].replace(',', '.'))
    name = df[surveyid].iloc[i]
    # Define the geometry of the area for which you would like images.
    aoi = ee.Geometry.Point(lon,lat)
    geom = ee.Geometry.Polygon(utils.point_to_box_coords(aoi=aoi,dimensionradius=IMAGE_DIMENSION))
    #create the neccessary Image Collections
    s2 = ee.ImageCollection('COPERNICUS/S2')
    s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
    s2 = s2.filterBounds(geom).filterDate(start_date,end_date).map(maskEdges)
    s2Clouds = s2Clouds.filterBounds(geom).filterDate(start_date,end_date)
    #Join S2 SR with cloud probability dataset to add cloud mask.
    filter = ee.Filter.equals(leftField='system:index', rightField='system:index')
    # Join S2 with cloud probability dataset to add cloud mask.
    s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
      primary =  s2, 
      secondary = s2Clouds, 
      condition = filter)
    maskCloudsWithProb = functools.partial(maskClouds, MAX_CLOUD_PROBABILITY = MAX_CLOUD_PROBABILITY)
    #create the median of each pixel to create one new image which is the export image
    s2 = ee.ImageCollection(s2SrWithCloudMask).map(maskCloudsWithProb).median()
    task_config = {
        'region': geom,
        'folder': DHS_EXPORT_FOLDER,
        'scale': 10,
        'crs': PROJECTION,
        'description': name
    }

    # Export Image
    task = ee.batch.Export.image.toDrive(s2, **task_config)
    #start the task
    task.start()